In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents


In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [3]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model='gpt-5-nano',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-5-nano',
    tools=[square]
)

## Calling subagents


In [11]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model='gpt-5-nano',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number. When the user asks for both the square root and the square of a number, you must call both call_subagent_1 (for square root) and call_subagent_2 (for square), then report both results clearly in your final answer.")

## Test


In [12]:
question = "What is the square root of 456 and what's the square of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

# Show the final answer (includes both square root and square)
print(response["messages"][-1].content)

Here are the results:

- Square root of 456: approximately 21.354156504062622
- Square of 456: 207,936

If you'd like, I can round the square root to a specific number of decimal places. How many decimals would you prefer?


In [13]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content="What is the square root of 456 and what's the square of 456?", additional_kwargs={}, response_metadata={}, id='bb1f9cff-ea7e-4f0c-8bbe-cc1154cd2e16'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 635, 'prompt_tokens': 259, 'total_tokens': 894, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D7VhggVipzk4BKjwSQpTcy3RAr7v8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c44e7-52ba-7993-a3cc-f959015a2dd2-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'call_kFEn3bsobYCWBQuyjLsZIyjP', 'type': 'tool_call'}, {'name': 'call_subagent_2', 'args'